In [1]:
import numpy as np

In [2]:
cluster_assignment = np.random.randint(0, 10, (50, 100))
print(cluster_assignment.shape)
n_x = np.zeros((50, 100, 20))
n_x[cluster_assignment == 5].shape

(50, 100)


(494, 20)

In [3]:
n_x.sum(axis = 2).shape

(50, 100)

In [4]:
# replace all elements of cluster_assignment that are equal to 5 with 0
cluster_assignment[cluster_assignment == 5] = 0

In [5]:
import numpy as np

# Your arrays
x = np.random.rand(50, 100)  # Replace with your actual data
y = np.random.rand(50, 100, 10)  # Replace with your actual data

# Assuming 'class' is an array of shape (50, 100) with integer class labels
classes = np.random.randint(0, 10, size=(50, 100))  # Replace with your actual class labels

# Mask for class == 5
class_mask = (classes == 5)

# Multiply elements in x by corresponding elements in y where class == 5
result = np.where(class_mask[..., None], x[..., None] * y, y)

In [6]:
result[:, :, :, None].shape

(50, 100, 10, 1)

In [7]:
1/ 3 * np.ones((3, 5))

array([[0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333],
       [0.33333333, 0.33333333, 0.33333333, 0.33333333, 0.33333333]])

In [8]:
np.sum(np.ones((1,3, 5)), axis = 2)[:, :, None]

array([[[5.],
        [5.],
        [5.]]])

In [9]:
class HistogramClustering(skl.base.BaseEstimator, skl.base.TransformerMixin):
    """Template class for HistogramClustering (HC)
    
    Attributes:
        centroids (np.ndarray): Array of centroid distributions p(y|c) with shape (n_clusters, n_bins).
        
    Parameters:
        n_clusters (int): Number of clusters (textures).
        n_bins (int): Number of bins used to discretize the range of pixel values found in input image X.
        window_size (int): Size of the window used to compute the local histograms for each pixel.
                           Should be an odd number larger or equal to 3.
        random_state (int): Random seed.
        estimation (str): Whether to use Maximum a Posteriori ("MAP") or
                          Deterministic Annealing ("DA") estimation.
    """
    
    def __init__(self, n_clusters=10, n_bins=64, window_size=7, random_state=42, estimation="MAP"):
        self.n_clusters = n_clusters
        self.n_bins =n_bins
        self.window_size = window_size
        self.random_state = random_state
        self.estimation = estimation
        # Add more parameters, if necessary.
        self.centroids = np.zeros((n_clusters, n_bins))
        self.n_iterations = 20
        
    def compute_histograms(self, X, window_size):
        rows, cols = X.shape
        empirical_dist = np.zeros((rows, cols, self.n_bins))
        n_x = np.zeros((rows, cols))
        
        # compute histograms for a window of size window_size around each pixel
        for i in range(rows):
            for j in range(cols):
                window = X[
                    max(0, i - (window_size - 1) // 2):min(rows, i + (window_size - 1) // 2 + 1), 
                    max(0, j - (window_size - 1) // 2):min(cols, j + (window_size - 1) // 2 + 1)
                ]
                n_x[i,j] = window.shape[0] * window.shape[1]
                hist, _ = np.histogram(window, bins=self.n_bins, range=(0, 1)) 
                empirical_dist[i, j, :] = hist / n_x[i, j] # n(x, y) / n(x)
        return empirical_dist, n_x
    

    # as in reference 1, we will try to make use of the spatial structure
    # and thus will update our prior
    def update_topological_prior(self, cluster_assignments, n_clusters, window_size):
        p_c = np.zeros((cluster_assignments.shape[0], cluster_assignments.shape[1], n_clusters))
        # sum kernel:
        kernel = np.ones((window_size, window_size))
        for c in range(n_clusters):
            cluster_mask = (cluster_assignments == c)
            p_c[:, :, c] = convolve2d(cluster_mask, kernel, mode='same', boundary='fill', fillvalue=0)
        # normalize prior by dividing by broadcasted sum
        p_c /= np.sum(p_c, axis = 2)[:, :, None]
        
        assert np.allclose(np.sum(p_c, axis = 2), 1)
        return p_c

    def update_cluster_assignment(self, p_y_given_c, p_y_given_x, p_c):
        # update cluster assignments
        nll = -p_y_given_x @ np.log(p_y_given_c.T  + 1e-6) -np.log(p_c + 1e-6)

        cluster_assignments = np.argmin(nll, axis = 2)
        return cluster_assignments

    
        
    
    def fit(self, X):
        """Compute HC for input image X
        
        Compute centroids.        
        
        Args:
            X (np.ndarray): Input array with shape (height, width)
        
        Returns:
            self
        """
        
        if self.estimation == "MAP":
            # as in the reference 1, we will use 1/N for p̂(x)
            p_x = 1 / (X.shape[0] * X.shape[1])
            # similarly, we also choose an uniform prior for p(c):
            p_c = 1 / self.n_clusters * np.ones((X.shape[0],X.shape[1],self.n_clusters))
            p_y_given_x, n_x = self.compute_histograms(X, self.window_size)
            
            # initialize p(y|c) as uniform; will be updated in EM like scheme
            p_y_given_c = np.zeros((self.n_clusters, self.n_bins))
            
            n_bins_per_cluster = self.n_bins // self.n_clusters
            
            for cluster in range(self.n_clusters):
                p_y_given_c[cluster, cluster * n_bins_per_cluster:(cluster + 1) * n_bins_per_cluster] = 1.0
                if cluster == self.n_clusters - 1:
                    # fill remaining bins with 1 for the last cluster
                    p_y_given_c[cluster, cluster * n_bins_per_cluster:] = 1.0
                    
            def compute_p_y_given_c(n_x, cluster_assignments, n_clusters, p_y_given_x):
                # p(y|c) i.e. the centroid distributions
                p_y_given_c = np.zeros((n_clusters, self.n_bins))
                
                for c in range(n_clusters):
                    cluster_mask = (cluster_assignments == c)
                    numerator = n_x[cluster_mask] @ p_y_given_x[cluster_mask]
                    denominator = np.sum(n_x[cluster_mask])
                    p_y_given_c[c, :] = numerator / (denominator + 1e-6)
                return p_y_given_c
                    
            
            
            for _ in range(self.n_iterations):
                cluster_assignments = self.update_cluster_assignment(p_y_given_c, p_y_given_x, p_c)
                p_c = self.update_topological_prior(cluster_assignments, self.n_clusters, self.window_size)
                p_y_given_c = compute_p_y_given_c(n_x, cluster_assignments, self.n_clusters, p_y_given_x)


                # TODO: add tolerance
            
            self.centroids = p_y_given_c
               
                
            
        elif self.estimation == "DA":
            pass
            
            # Code for Deterministic Annealing estimation
        
        return self
    
    def predict(self, X):
        """Predict cluster assignments for each pixel in image X.
        
        Args:
            X (np.ndarray): Input array with shape (height, width)
            
        Returns:
            C (np.ndarray): Assignment map (height, width) 
        """
        check_is_fitted(self, ["centroids"])
        
        # uniform prior for cluster assignments
        p_c = 1 / self.n_clusters * np.ones((X.shape[0],X.shape[1],self.n_clusters))

        # empirical probability distribution:
        p_y_given_x, _ = self.compute_histograms(X, self.window_size)
        
        for _ in range(self.n_iterations):
                C = self.update_cluster_assignment(self.centroids, p_y_given_x, p_c)
                p_c = self.update_topological_prior(C, self.n_clusters, self.window_size)

        return C
    
    def generate(self, C):
        """Generate a sample image X from a texture label map C.
        
        The entries of C are integers from the set {1,...,n_clusters}. They represent the texture labels
        of each pixel. Given the texture labels, a sample image X is generated by sampling
        the value of each pixel from the fitted p(y|c).
        
        Args:
            C (np.ndarray): Input array with shape (height, width)
            
        Returns:
            X (np.ndarray): Sample image (height, width)
        """
        check_is_fitted(self, ["centroids"])
        
        # Your code goes here
        
        return X

NameError: name 'skl' is not defined

In [ ]:
_, bins = np.histogram(1, bins=64, range=(0, 1))

In [ ]:
bins.shape

(65,)

In [ ]:
bins

array([0.      , 0.015625, 0.03125 , 0.046875, 0.0625  , 0.078125,
       0.09375 , 0.109375, 0.125   , 0.140625, 0.15625 , 0.171875,
       0.1875  , 0.203125, 0.21875 , 0.234375, 0.25    , 0.265625,
       0.28125 , 0.296875, 0.3125  , 0.328125, 0.34375 , 0.359375,
       0.375   , 0.390625, 0.40625 , 0.421875, 0.4375  , 0.453125,
       0.46875 , 0.484375, 0.5     , 0.515625, 0.53125 , 0.546875,
       0.5625  , 0.578125, 0.59375 , 0.609375, 0.625   , 0.640625,
       0.65625 , 0.671875, 0.6875  , 0.703125, 0.71875 , 0.734375,
       0.75    , 0.765625, 0.78125 , 0.796875, 0.8125  , 0.828125,
       0.84375 , 0.859375, 0.875   , 0.890625, 0.90625 , 0.921875,
       0.9375  , 0.953125, 0.96875 , 0.984375, 1.      ])

In [ ]:
o = np.ones((50, 100))
int(np.unique(o))


1

In [ ]:
x = np.zeros((3,2))
x[:, 1] = np.random.uniform(0, 1, size = 3)

In [24]:
np.random.seed(43)
a = np.random.rand(50, 20, 30)
b = np.random.randn(10, 30)
a / b[5, :]

array([[[-0.38233049,  0.18632102, -0.0820157 , ..., -1.56423435,
          1.09135967,  0.15601819],
        [-0.01704568,  0.1661724 , -0.29242138, ..., -1.22878397,
          2.23235316,  0.32121102],
        [-0.68481746,  0.06092597, -0.48926164, ..., -0.99323092,
          0.0186162 ,  0.36358624],
        ...,
        [-3.28981602,  0.02600658, -0.4285244 , ..., -0.6710291 ,
          1.72775292,  0.12779547],
        [-0.03136888,  0.11149477, -0.07972406, ..., -0.3221038 ,
          3.08612266,  0.19033898],
        [-0.92819196,  0.08670564, -0.31001949, ..., -0.17666736,
          1.71058793,  0.5124434 ]],

       [[-1.78636784,  0.18293435, -0.38706202, ..., -0.94477197,
          0.82522134,  0.03847765],
        [-1.04607894,  0.16467855, -0.38470686, ..., -0.2299643 ,
          3.31857784,  0.20160222],
        [-1.30170655,  0.10953434, -0.53664315, ..., -0.39330205,
          3.28159769,  0.39183743],
        ...,
        [-2.91062473,  0.08638886, -0.21025233, ..., -

In [44]:
a = np.random.rand(100, 200, 20)
b = np.random.rand(100, 200, 20)
print(np.tensordot(b, a, axes = ([0, 1], [0,1])).shape)
print(np.tensordot(a,b, axes = ([0,1,2], [0,1,2])).shape)

(20, 20)
()


In [46]:
a = np.random.rand(100, 20, 1)
b = np.random.rand(100, 20)
a[:, :, None]*b

array([[[[1.75318801e-01, 1.27248948e-01, 6.02067790e-02, ...,
          5.74923305e-02, 4.18467680e-03, 4.69788352e-01],
         [3.11470437e-01, 7.32683729e-01, 1.70601593e-01, ...,
          4.11110034e-01, 2.65358789e-01, 8.83184029e-02],
         [7.04744444e-01, 9.00067340e-02, 2.84772281e-01, ...,
          1.02153082e-01, 7.32970964e-01, 4.26088881e-01],
         ...,
         [4.50404682e-02, 7.89616517e-01, 5.30903451e-01, ...,
          1.26347814e-01, 1.02186425e-01, 3.90324205e-01],
         [1.59100243e-01, 5.07969663e-01, 7.92596687e-01, ...,
          4.06119695e-02, 1.89382192e-01, 1.54062152e-01],
         [2.17819540e-01, 7.72064520e-01, 3.27139675e-01, ...,
          5.46455849e-02, 2.97217426e-01, 5.53146939e-01]],

        [[1.02319512e-01, 7.42649967e-02, 3.51378642e-02, ...,
          3.35536585e-02, 2.44225995e-03, 2.74177752e-01],
         [1.81780293e-01, 4.27608682e-01, 9.95664561e-02, ...,
          2.39931928e-01, 1.54868625e-01, 5.15443626e-02],
        